In [35]:
import pandas as pd
import geopandas as gpd
import arcpy
import os
import shutil
import math
import inspect

src_file_path = inspect.getfile(lambda: None)
THIS_FOLDER = os.path.dirname(os.path.abspath(src_file_path))

In [36]:
arcpy.ResetEnvironments()
gdb_name = "Summarize_Reporting_Units.gdb"
arcpy.env.overwriteOutput = True
path = os.path.join(THIS_FOLDER, gdb_name)
# arcpy.CreateFileGDB_management("./", gdb_name)
arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path

In [37]:
data = ["in_data/Pr_Lg_Fire.shp", "in_data/Pr_Fire_Ign.shp",
        "in_data/Pct_Cg_Lg_Fi.shp", "in_data/Pct_Cg_Fi_Ign.shp",
        "in_data/WUI_Intermix.shp", "in_data/WUI_Interface.shp",
        "in_data/FloraFaunaMerge.shp",
        "out_data/lu/LU_merged.shp", "out_data/ru/Reporting_Units_APN.shp",
        "out_data/building/Building_Intersect.shp"]

for v in data:
    path = os.path.join(THIS_FOLDER, "{}".format(v))
    name = v.rsplit('/', 1)[-1].split(".")[0]
    arcpy.CopyFeatures_management(path, name)
    crs = arcpy.SpatialReference('NAD 1983 StatePlane California V FIPS 0405 (US Feet)')
    arcpy.Project_management(name, "{}_rpj".format(name), crs)

In [38]:
arcpy.analysis.SummarizeWithin("Reporting_Units_APN_rpj", "Pr_Lg_Fire_rpj", 
                               "A_RU_1", "KEEP_ALL", "gridcode Mean", 
                               "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_1
1,


In [39]:
arcpy.analysis.SummarizeWithin("A_RU_1", "Pr_Fire_Ign_rpj", 
                               "A_RU_2", "KEEP_ALL", "gridcode Mean", 
                               "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_2
1,


In [40]:
arcpy.analysis.SummarizeWithin("A_RU_2", "Pct_Cg_Lg_Fi_rpj", 
                               "A_RU_3", "KEEP_ALL", "gridcode Mean", 
                               "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_3
1,


In [41]:
arcpy.analysis.SummarizeWithin("A_RU_3", "Pct_Cg_Fi_Ign_rpj", 
                               "A_RU_4", "KEEP_ALL", "gridcode Mean", 
                               "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_4
1,


In [42]:
arcpy.analysis.SummarizeWithin("A_RU_4", "WUI_Intermix_rpj", 
                               "A_RU_5", "KEEP_ALL", None, 
                               "ADD_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)
arcpy.analysis.SummarizeWithin("A_RU_5", "WUI_Interface_rpj", 
                               "A_RU_6", "KEEP_ALL", None, 
                               "ADD_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_6
1,


In [43]:
arcpy.analysis.SummarizeWithin("A_RU_6", "LU_Merged_rpj", 
                               "A_RU_7", "KEEP_ALL", "LU_Cat Max", 
                               "NO_SHAPE_SUM", "SQUAREFEET", "LU_Cat", "ADD_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_7
1,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\LU_Cat_Summary


In [44]:
arcpy.analysis.SummarizeWithin("A_RU_7", "FloraFaunaMerge_rpj", 
                               "A_RU_8", "KEEP_ALL", "Flora_Faun Mean", 
                               "NO_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_8
1,


In [49]:
arcpy.analysis.SummarizeWithin("A_RU_8", "Building_Intersect_rpj", 
                               "A_RU_9", "KEEP_ALL", "", 
                               "ADD_SHAPE_SUM", "SQUAREFEET", None, "NO_MIN_MAJ", "NO_PERCENT", None)

id,value
0,C:\Users\zrcan\Documents\GitHub\RPP-SBC-EEMS\Summarize_Reporting_Units.gdb\A_RU_9
1,


In [50]:
fieldObs = arcpy.ListFields("A_RU_9")  

fieldNames = []  
for field in fieldObs:  
    fieldNames.append(field.name)  
del fieldObs  
fieldCount = len(fieldNames) 

with arcpy.da.UpdateCursor("A_RU_9", fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0"  
        curU.updateRow(rowU)

del curU

In [52]:
codeblock = """
def get_val(m):
    if m > 0:
        return 1
    else:
        return 0
"""

# Execute CalculateField 
arcpy.CalculateField_management("A_RU_9", "WUI_Interm", "get_val(!SUM_Area_SQUAREFEET!)", "PYTHON3", codeblock)
arcpy.CalculateField_management("A_RU_9", "WUI_Interf", "get_val(!SUM_Area_SQUAREFEET_1!)", "PYTHON3", codeblock)

arcpy.DeleteField_management("A_RU_9", 
                             ['SUM_Area_SQUAREFEET', 'SUM_Area_SQUAREFEET_1'])

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Summarize_Reporting_Units.gdb\\A_RU_9'>

In [53]:
fieldObs = arcpy.ListFields("A_RU_9") 

fieldNames = []  
for field in fieldObs: 
    print(field.name)
    fieldNames.append(field.name)  
del fieldObs

OBJECTID
Shape
Join_ID
apn
Shape_Leng
MEAN_gridcode
MEAN_gridcode_1
MEAN_gridcode_12
MEAN_gridcode_12_13
SUM_Area_SQUAREFEET_12
MAX_LU_Cat
Minority_LU_Cat
Majority_LU_Cat
Join_ID_1
MEAN_Flora_Faun
Shape_Length
Shape_Area
SUM_Area_SQUAREFEET_12_13
WUI_Interm
WUI_Interf


In [55]:
arcpy.AlterField_management('A_RU_9', 
                            'MEAN_gridcode', 'Pr_Lg_Fr', 'Pr_Lg_Fr')
arcpy.AlterField_management('A_RU_9', 
                            'MEAN_gridcode_1', 'Pr_Fi_Ign', 'Pr_Fi_Ign')
arcpy.AlterField_management('A_RU_9', 
                            'MEAN_gridcode_12', 'Pc_Cg_LgFi', 'Pc_Cg_LgFi')
arcpy.AlterField_management('A_RU_9', 
                            'MEAN_gridcode_12_13', 'Pc_Fi_Ign', 'Pc_Fi_Ign')

arcpy.AlterField_management('A_RU_9', 
                            'MEAN_Flora_Faun', 'Fl_Fa', 'Fl_Fa')

arcpy.AlterField_management('A_RU_9', 
                            'SUM_Area_SQUAREFEET_12_13', 'Buil_Ar', 'Buil_Ar')

arcpy.AlterField_management('A_RU_9', 
                            'Majority_LU_Cat', 'Mj_LU_Cat', 'Mj_LU_Cat')
arcpy.DeleteField_management("A_RU_9", 
                             ['SUM_Area_SQUAREFEET_12', 'MAX_LU_Cat', 'Minority_LU_Cat', 'Join_ID_1', 'Shape_Leng'])

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Summarize_Reporting_Units.gdb\\A_RU_9'>

In [56]:
arcpy.AddField_management("A_RU_9", "Buil_Rat", "FLOAT")

arcpy.CalculateField_management("A_RU_9", "Buil_Rat", 
                                '!Buil_Ar!/!Shape_Area!', "PYTHON3")

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\Summarize_Reporting_Units.gdb\\A_RU_9'>

In [57]:
dir_path = os.path.join(THIS_FOLDER, "out_data/eems")
shutil.rmtree(dir_path)
os.mkdir(dir_path)

arcpy.conversion.FeatureClassToShapefile("A_RU_9", 
                                        os.path.join(THIS_FOLDER, "out_data/eems"))

<Result 'C:\\Users\\zrcan\\Documents\\GitHub\\RPP-SBC-EEMS\\out_data\\eems'>

In [58]:
gdf = gpd.read_file(os.path.join(THIS_FOLDER, "out_data/eems/A_RU_9.shp"))
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 18180 entries, 0 to 18179
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Join_ID     18180 non-null  int64   
 1   apn         18179 non-null  object  
 2   Pr_Lg_Fr    18180 non-null  float64 
 3   Pr_Fi_Ign   18180 non-null  float64 
 4   Pc_Cg_LgFi  18180 non-null  float64 
 5   Pc_Fi_Ign   18180 non-null  float64 
 6   Mj_LU_Cat   18180 non-null  object  
 7   Fl_Fa       18180 non-null  float64 
 8   Shape_Leng  18180 non-null  float64 
 9   Shape_Area  18180 non-null  float64 
 10  Buil_Ar     18180 non-null  float64 
 11  WUI_Interm  18180 non-null  object  
 12  WUI_Interf  18180 non-null  object  
 13  Buil_Rat    18180 non-null  float64 
 14  geometry    18180 non-null  geometry
dtypes: float64(9), geometry(1), int64(1), object(4)
memory usage: 2.1+ MB


In [59]:
gdf.to_csv("out_data/eems/A_RU_9.csv", index=False)